* original code from https://lsjsj92.tistory.com/568

In [3]:
import numpy as np
import pandas as pd

from database import fetch_data_music_and_groups, fetch_artist, fetch_theme, close_mysql_connection

In [4]:
musics = fetch_data_music_and_groups()
column_names = ['music_id', 'title', 'genre', 'lyrics', 'release_date', 'likes', 'view',
                'highest_pitch', 'lowest_pitch', 'high_pitch_ratio', 'low_pitch_ratio',
                'group_id', 'group_name', 'group_type']

musics.columns = column_names

musics

2024-10-28 02:12:03.438978: [SQL] SELECT m.music_id, m.title, m.genre, m.lyrics, m.release_date, m.likes, m.`view`, ma.highest_pitch, ma.lowest_pitch, ma.high_pitch_ratio, ma.low_pitch_ratio, g.group_id, g.group_name, g.group_type FROM (music m NATURAL JOIN music_analysis ma) NATURAL JOIN groups g WHERE ma.`status` = 'COMPLETE'


,music_id,title,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,group_id,group_name,group_type
0,169,야생화,BALLADE,하얗게 피어난 얼음 꽃 하나가\n달가운 바람에 얼굴을 내밀어\n아무 말 못했던 이름...,2014-03-28,2,42,926.378401,125.573964,0.199561,0.000000,11,박효신,SOLO
1,170,"너에게 난, 나에게 넌",FOLK,너에게 난 해질녘 노을처럼\n한 편의 아름다운 추억이 되고\n소중했던 우리 푸르던 ...,2003-02-01,0,36,944.986819,140.076631,0.397028,0.000000,13,자전거 탄 풍경,GROUP
2,171,너였다면,OST,왜 너에겐 그렇게 어려운지\n애를 쓰는 나를\n제대로 봐주는 게\n너 하나에 이토록...,2016-05-31,0,25,513.832400,145.657815,0.345133,0.000000,12,마크툽 (MAKTUB),SOLO
3,172,시작,OST,새로운 시작은 늘 설레게 하지\n모든 걸 이겨낼 것처럼\n시간을 뒤쫓는 시계바늘처럼...,2020-02-01,0,8,428.380936,149.105775,0.331224,0.000000,14,가호 (Gaho),SOLO
4,173,어땠을까 (Feat. 박정현),RAP,내가 그때 널 잡았더라면\n너와 나 지금보다 행복했을까\n마지막에 널\n안아줬다면 ...,2012-07-15,1,9,620.806132,123.754295,0.485465,0.000000,15,싸이 (PSY),SOLO
5,174,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,OST,흔들리는 꽃들 속에서\n네 샴푸향이 느껴진거야\n스쳐지나간건가 뒤돌아보지만\n그냥 ...,2019-08-23,0,8,500.865280,104.255744,0.052632,0.005848,16,장범준,SOLO
6,175,Don't Cry,BALLADE,그대의 눈물이 마를 때까지\n우리의 사랑을 볼 수 없을테니\n울지 말아요 이제야\n...,2003-08-01,0,4,796.514962,124.552403,0.440945,0.000000,17,더 크로스,GROUP
7,176,잠시만 안녕,BALLADE,행복을 줄 수 없었어\n그런데 사랑을 했어\n니곁에 감히 머무른\n내 욕심을 용서치...,2007-05-10,0,12,885.293572,166.179307,0.493438,0.000000,18,엠씨더맥스 (M.C the MAX),SOLO
8,177,인형의 꿈,OST,그대 먼 곳만 보네요\n내가 바로 여기 있는데\n조금만 고개를 돌려도\n날 볼 수 ...,2003-09-09,1,5,588.248207,232.075376,0.021583,0.120504,19,러브홀릭,SOLO
9,178,외톨이야,ROCK,외톨이야 외톨이야\n\n외톨이야 외톨이야\n\n\n\n봐봐 나를 봐봐 똑바로 내 두...,2010-01-14,0,4,557.989885,150.122581,0.468354,0.000000,20,CNBLUE (씨엔블루),GROUP


In [5]:
artists = fetch_artist()
column_names = ['group_id', 'artist_id', 'artist_name', 'gender', 'nation']
artists.columns = column_names

artists

2024-10-28 02:12:05.035933: [SQL] SELECT gai.group_id, a.artist_id, a.artist_name, a.gender, a.nation FROM (SELECT g.group_id, gm.artist_id FROM groups g NATURAL JOIN group_member gm) gai    INNER JOIN artist a ON gai.artist_id = a.artist_id


,group_id,artist_id,artist_name,gender,nation
0,7,2,이영지,FEMALE,대한민국
1,11,11,박효신,MALE,대한민국
2,13,13,강인봉,MALE,대한민국
3,13,14,김형섭,MALE,대한민국
4,13,15,송봉주,MALE,대한민국
...,...,...,...,...,...
106,76,152,윤미래,FEMALE,미국
107,77,153,성진,MALE,대한민국
108,77,154,Young K (DAY6),MALE,대한민국
109,77,155,원필 (DAY6),MALE,대한민국


In [6]:
artist_dict = {}

for at in artists.itertuples():
    if at.group_id in artist_dict:
        artist_dict[at.group_id].append(at)
    else:
        artist_dict[at.group_id] = [at]

In [5]:
artist_dict[42]

[Pandas(Index=38, group_id=42, artist_id=75, artist_name='소연 ((여자)아이들)', gender='FEMALE', nation='대한민국'),
 Pandas(Index=39, group_id=42, artist_id=76, artist_name='미연 ((여자)아이들)', gender='FEMALE', nation='대한민국'),
 Pandas(Index=40, group_id=42, artist_id=77, artist_name='우기 ((여자)아이들)', gender='FEMALE', nation='중국'),
 Pandas(Index=41, group_id=42, artist_id=78, artist_name='슈화 ((여자)아이들)', gender='FEMALE', nation='타이완'),
 Pandas(Index=42, group_id=42, artist_id=79, artist_name='민니 ((여자)아이들)', gender='FEMALE', nation='태국')]

In [6]:
# 가수 관련 열 추가

In [7]:
def classify_gender(gender_list):
    # 'OTHER'이 하나라도 있으면 '기타'
    if 'OTHER' in gender_list:
        return '기타'
    
    # 'FEMALE'과 'MALE'이 섞여 있으면 '혼성'
    if 'FEMALE' in gender_list and 'MALE' in gender_list:
        return '혼성'
    
    # 'FEMALE'만 있으면 '여성'
    if all(g == 'FEMALE' for g in gender_list):
        return '여성'
    
    # 'MALE'만 있으면 '남성'
    if all(g == 'MALE' for g in gender_list):
        return '남성'
    
    # 예외 처리를 위해 기본 반환값 (조건에 맞지 않는 경우)
    return '기타'

def classify_nation(nation_list):
    # '대한민국' 또는 '한국'이 하나라도 있으면 '대한민국' 반환
    if '대한민국' in nation_list or '한국' in nation_list:
        return '대한민국'
    
    # 중복 제거한 국가 목록
    unique_nations = set(nation_list)
    
    # 하나의 국가만 있으면 그 국가를 반환
    if len(unique_nations) == 1:
        return unique_nations.pop()
    
    # 여러 국가가 섞여 있으면 '다국가' 반환
    return '다국가'

In [8]:
' '.join(at.artist_name for at in artist_dict[42])

'소연 ((여자)아이들) 미연 ((여자)아이들) 우기 ((여자)아이들) 슈화 ((여자)아이들) 민니 ((여자)아이들)'

In [9]:
[at.gender for at in artist_dict[42]]

['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']

In [10]:
# 가수 이름 열 추가
musics['artist_names'] = musics.apply(lambda row: '|'.join(at.artist_name for at in artist_dict[row.group_id]), axis=1)

# 노래 성별(남자, 여자, 혼성, 그외) 구분 추가
musics['gender'] = musics.apply(lambda row: classify_gender([at.gender for at in artist_dict[row.group_id]]), axis=1)

# 국가 열 추가
musics['nation'] = musics.apply(lambda row: classify_nation([at.nation for at in artist_dict[row.group_id]]), axis=1)

In [11]:
# 테마
themes = fetch_theme()
column_names = ['music_id', 'theme_name']
themes.columns = column_names

themes

2024-10-28 02:12:22.442431: [SQL] SELECT m.music_id, t.theme_name FROM music m INNER JOIN theme t ON m.music_id = t.music_id


,music_id,theme_name
0,169,고음
1,169,저음
2,205,고음


In [12]:
theme_dict = {}

for tm in themes.itertuples():
    if tm.music_id in theme_dict:
        theme_dict[tm.music_id] = theme_dict[tm.music_id]+'|'+tm.theme_name
    else:
        theme_dict[tm.music_id] = tm.theme_name

In [13]:
theme_dict[169]

'고음|저음'

In [14]:
# 테마 열 추가
musics['themes'] = musics.apply(lambda row: theme_dict.get(row.music_id, ''), axis=1)

In [15]:
musics

,music_id,title,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,group_id,group_name,group_type,artist_names,gender,nation,themes
0,169,야생화,BALLADE,하얗게 피어난 얼음 꽃 하나가\n달가운 바람에 얼굴을 내밀어\n아무 말 못했던 이름...,2014-03-28,2,42,926.378401,125.573964,0.199561,0.000000,11,박효신,SOLO,박효신,남성,대한민국,고음|저음
1,170,"너에게 난, 나에게 넌",FOLK,너에게 난 해질녘 노을처럼\n한 편의 아름다운 추억이 되고\n소중했던 우리 푸르던 ...,2003-02-01,0,36,944.986819,140.076631,0.397028,0.000000,13,자전거 탄 풍경,GROUP,강인봉|김형섭|송봉주,남성,대한민국,
2,171,너였다면,OST,왜 너에겐 그렇게 어려운지\n애를 쓰는 나를\n제대로 봐주는 게\n너 하나에 이토록...,2016-05-31,0,25,513.832400,145.657815,0.345133,0.000000,12,마크툽 (MAKTUB),SOLO,마크툽 (MAKTUB),남성,대한민국,
3,172,시작,OST,새로운 시작은 늘 설레게 하지\n모든 걸 이겨낼 것처럼\n시간을 뒤쫓는 시계바늘처럼...,2020-02-01,0,8,428.380936,149.105775,0.331224,0.000000,14,가호 (Gaho),SOLO,가호 (Gaho),남성,대한민국,
4,173,어땠을까 (Feat. 박정현),RAP,내가 그때 널 잡았더라면\n너와 나 지금보다 행복했을까\n마지막에 널\n안아줬다면 ...,2012-07-15,1,9,620.806132,123.754295,0.485465,0.000000,15,싸이 (PSY),SOLO,싸이 (PSY),남성,대한민국,
5,174,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,OST,흔들리는 꽃들 속에서\n네 샴푸향이 느껴진거야\n스쳐지나간건가 뒤돌아보지만\n그냥 ...,2019-08-23,0,8,500.865280,104.255744,0.052632,0.005848,16,장범준,SOLO,장범준,남성,대한민국,
6,175,Don't Cry,BALLADE,그대의 눈물이 마를 때까지\n우리의 사랑을 볼 수 없을테니\n울지 말아요 이제야\n...,2003-08-01,0,4,796.514962,124.552403,0.440945,0.000000,17,더 크로스,GROUP,SiHa (이시하)|김혁건(The Cross),남성,대한민국,
7,176,잠시만 안녕,BALLADE,행복을 줄 수 없었어\n그런데 사랑을 했어\n니곁에 감히 머무른\n내 욕심을 용서치...,2007-05-10,0,12,885.293572,166.179307,0.493438,0.000000,18,엠씨더맥스 (M.C the MAX),SOLO,이수 (엠씨더맥스),남성,대한민국,
8,177,인형의 꿈,OST,그대 먼 곳만 보네요\n내가 바로 여기 있는데\n조금만 고개를 돌려도\n날 볼 수 ...,2003-09-09,1,5,588.248207,232.075376,0.021583,0.120504,19,러브홀릭,SOLO,지선 (Loveholic),여성,대한민국,
9,178,외톨이야,ROCK,외톨이야 외톨이야\n\n외톨이야 외톨이야\n\n\n\n봐봐 나를 봐봐 똑바로 내 두...,2010-01-14,0,4,557.989885,150.122581,0.468354,0.000000,20,CNBLUE (씨엔블루),GROUP,이정신 (CNBLUE)|강민혁 (CNBLUE)|정용화,남성,대한민국,


In [16]:
from konlpy.tag import Okt
from collections import Counter

okt = Okt()

def remove_lyric_stop_words(lyrics):
    if lyrics:
        tags = okt.pos(lyrics, norm=True, stem=True)
    
        # 조사 제거
        tags = list(filter(lambda t: t[1] == 'Noun', tags))
        # 줄바꿈 문자 제거
        tags = list(filter(lambda t: not t[0].startswith('\n'), tags))
        
        # 단어만 추출하고, 2글자 이상인 단어 필터링
        words = [word for word, pos in tags if len(word) >= 2]
        
        # 단어 개수 세기
        word_counts = Counter(words)
        
        # 개수가 가장 많은 5개의 단어 추출
        most_common_words = word_counts.most_common(10)
        
        return ' '.join([tag[0] for tag in most_common_words])
        
    else: # 가사 없는 경우
        return ''

In [17]:
remove_lyric_stop_words(musics.lyrics[0])

'눈물 바람 한번 기억 마음 남아 먹음 다시 피어 얼음'

In [18]:
# 가사 형태소 분석 후 명사와 동사만 남기기

musics['lyrics'] = musics.apply(lambda row: remove_lyric_stop_words(row.lyrics), axis=1)
musics

,music_id,title,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,group_id,group_name,group_type,artist_names,gender,nation,themes
0,169,야생화,BALLADE,눈물 바람 한번 기억 마음 남아 먹음 다시 피어 얼음,2014-03-28,2,42,926.378401,125.573964,0.199561,0.000000,11,박효신,SOLO,박효신,남성,대한민국,고음|저음
1,170,"너에게 난, 나에게 넌",FOLK,남아 노을 추억 우리 기억 그림 주기 한편 우후 영원,2003-02-01,0,36,944.986819,140.076631,0.397028,0.000000,13,자전거 탄 풍경,GROUP,강인봉|김형섭|송봉주,남성,대한민국,
2,171,너였다면,OST,하루 가슴 사랑 대답 혼자 가득 채운 통증 얼마나 수도,2016-05-31,0,25,513.832400,145.657815,0.345133,0.000000,12,마크툽 (MAKTUB),SOLO,마크툽 (MAKTUB),남성,대한민국,
3,172,시작,OST,시작 지금 대로 그게 바로 꿈일 버티고 다시 모든 시간,2020-02-01,0,8,428.380936,149.105775,0.331224,0.000000,14,가호 (Gaho),SOLO,가호 (Gaho),남성,대한민국,
4,173,어땠을까 (Feat. 박정현),RAP,사랑 지금 그때 마지막 혼자 기억 우리 사람 그땐 아무,2012-07-15,1,9,620.806132,123.754295,0.485465,0.000000,15,싸이 (PSY),SOLO,싸이 (PSY),남성,대한민국,
5,174,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,OST,연락 항상 지금 집앞 그냥 계속 샴푸 건가 마음 사람,2019-08-23,0,8,500.865280,104.255744,0.052632,0.005848,16,장범준,SOLO,장범준,남성,대한민국,
6,175,Don't Cry,BALLADE,사랑 안녕 그대 세월 눈물 우리 이제야 아픔 상처 약속,2003-08-01,0,4,796.514962,124.552403,0.440945,0.000000,17,더 크로스,GROUP,SiHa (이시하)|김혁건(The Cross),남성,대한민국,
7,176,잠시만 안녕,BALLADE,안녕 사랑 행복 자격 잠시 위해 감히 욕심 용서 치마,2007-05-10,0,12,885.293572,166.179307,0.493438,0.000000,18,엠씨더맥스 (M.C the MAX),SOLO,이수 (엠씨더맥스),남성,대한민국,
8,177,인형의 꿈,OST,그대 걸음 항상 모습 손짓 사랑 바로 여기 조금 고개,2003-09-09,1,5,588.248207,232.075376,0.021583,0.120504,19,러브홀릭,SOLO,지선 (Loveholic),여성,대한민국,
9,178,외톨이야,ROCK,외톨이야 사랑 다른 사람 눈물 외톨이 요즘 가슴 마음 차라리,2010-01-14,0,4,557.989885,150.122581,0.468354,0.000000,20,CNBLUE (씨엔블루),GROUP,이정신 (CNBLUE)|강민혁 (CNBLUE)|정용화,남성,대한민국,


In [19]:
musics_str = 'genre:' + musics.genre
musics_str = musics_str + ' group_id:' + musics.group_id.astype(str)
musics_str = musics_str + ' group_type:' + musics.group_type

musics_str = musics_str + ' ' + musics.artist_names.apply(
    lambda x: ' '.join(['artist:' + artist.replace(' ', '_') for artist in x.split('|')])
)

musics_str = musics_str + ' gender:' + musics.gender
musics_str = musics_str + ' nation:' + musics.nation

musics_str = musics_str + ' ' + musics.themes.apply(
    lambda x: ' '.join(['theme:' + theme.replace(' ', '_') if theme else '' for theme in x.split('|')])
)

musics_str = musics_str + ' ' + ' hPitch:' + musics.highest_pitch.apply(lambda x: str(round(x / 100) * 100))
musics_str = musics_str + ' ' + ' lPitch:' + musics.lowest_pitch.apply(lambda x: str(round(x / 100) * 100))
musics_str = musics_str + ' ' + ' hRatio:' + musics.high_pitch_ratio.apply(lambda x: str(round(round(x * 100) / 10)* 10))
musics_str = musics_str + ' ' + ' lRatio:' + musics.low_pitch_ratio.apply(lambda x: str(round(round(x * 100) / 10)* 10))

musics_str = musics_str + ' ' + musics.lyrics

musics_str

0     genre:BALLADE group_id:11 group_type:SOLO arti...
1     genre:FOLK group_id:13 group_type:GROUP artist...
2     genre:OST group_id:12 group_type:SOLO artist:마...
3     genre:OST group_id:14 group_type:SOLO artist:가...
4     genre:RAP group_id:15 group_type:SOLO artist:싸...
5     genre:OST group_id:16 group_type:SOLO artist:장...
6     genre:BALLADE group_id:17 group_type:GROUP art...
7     genre:BALLADE group_id:18 group_type:SOLO arti...
8     genre:OST group_id:19 group_type:SOLO artist:지...
9     genre:ROCK group_id:20 group_type:GROUP artist...
10    genre:ROCK group_id:21 group_type:SOLO artist:...
11    genre:BALLADE group_id:22 group_type:SOLO arti...
12    genre:FOLK group_id:23 group_type:SOLO artist:...
13    genre:BALLADE group_id:24 group_type:GROUP art...
14    genre:RNB group_id:25 group_type:SOLO artist:휘...
15    genre:DANCE group_id:35 group_type:SOLO artist...
16    genre:RNB group_id:24 group_type:GROUP artist:...
17    genre:BALLADE group_id:36 group_type:GROUP

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def custom_tokenizer(text):
    tokens = text.split()
    weighted_tokens = []
    for token in tokens:
        if token.startswith("genre:"):
            weighted_tokens.append((token, 20))  # 장르 가중치 20
        elif token.startswith("group_id:"):
            weighted_tokens.append((token, 20))  # 그룹 가중치 20
        elif token.startswith("group_type:"):
            weighted_tokens.append((token, 10))  # 그룹 형태 가중치 10
        elif token.startswith("artist:"):
            weighted_tokens.append((token, 30))  # 가수 명 가중치 30
        elif token.startswith("gender:"):
            weighted_tokens.append((token, 60))  # 성별 가중치 60
        elif token.startswith("nation:"):
            weighted_tokens.append((token, 40))  # 국가 가중치 40
        elif token.startswith("theme:"):
            weighted_tokens.append((token, 10))  # 테마 가중치 10
        elif token.startswith("hPitch:"):
            weighted_tokens.append((token, 10))  # 최고음 가중치 10
        elif token.startswith("lPitch:"):
            weighted_tokens.append((token, 10))  # 최저음 가중치 10
        elif token.startswith("hRatio:"):
            weighted_tokens.append((token, 10))  # 고음 비율 가중치 10
        elif token.startswith("lRatio:"):
            weighted_tokens.append((token, 10))  # 저음 비율 가중치 10
        else:
            weighted_tokens.append((token, 1))  # 가사 가중치 1
    return weighted_tokens    

def custom_preprocessor(text):
    expanded_tokens = [" ".join([token] * weight) for token, weight in custom_tokenizer(text)]
    return " ".join(expanded_tokens)

tfidf_vector = TfidfVectorizer(token_pattern=r"\b\S+\b", tokenizer=None, preprocessor=custom_preprocessor)
tfidf_matrix = tfidf_vector.fit_transform(musics_str).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()

In [24]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = musics.title)
print(tfidf_matrix.shape)
#tfidf_matrix.head()

(55, 477)


In [22]:
row_index = 0  # 정렬할 행의 인덱스
sorted_row = tfidf_matrix.iloc[row_index].sort_values(ascending=False)
sorted_row

artist:박효신       0.625931
group_id:11      0.417287
gender:남성        0.408724
genre:BALLADE    0.272411
nation:대한민국      0.248065
                   ...   
group_id:54      0.000000
group_id:53      0.000000
group_id:52      0.000000
group_id:51      0.000000
후회               0.000000
Name: 야생화, Length: 477, dtype: float64

In [23]:
# 유사도 구하기

cosine_sim = cosine_similarity(tfidf_matrix)

In [24]:
cosine_sim.shape

(38, 38)

In [25]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = musics.title, columns = musics.title)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(38, 38)


title,야생화,"너에게 난, 나에게 넌",너였다면,시작,어땠을까 (Feat. 박정현),흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,Don't Cry,잠시만 안녕,인형의 꿈,외톨이야,...,저곳으로(인어공주OST),See You Again (분노의 질주 : 더 세븐 OST),사랑은 늘 도망가,"모든 날, 모든 순간",부동의 첫사랑,FRIEND THE END,주저하는 연인들을 위해,친구 얘긴데 (feat. Skinny Brown),첫사랑,오늘도 빛나는 너에게
title,,,,,,,,,,,,,,,,,,,,,
야생화,1.000000,0.189866,0.288160,0.269515,0.240874,0.255166,0.275857,0.345503,0.051101,0.173269,...,0.054518,0.139127,0.255158,0.246726,0.269504,0.054152,0.220019,0.252769,0.084715,0.283050
"너에게 난, 나에게 넌",0.189866,1.000000,0.198723,0.185702,0.190406,0.186918,0.160803,0.205978,0.034367,0.146019,...,0.036894,0.119835,0.186945,0.180638,0.185708,0.036646,0.161973,0.185424,0.036303,0.207573
너였다면,0.288160,0.198723,1.000000,0.378862,0.269188,0.363342,0.247087,0.273992,0.123708,0.193486,...,0.128140,0.196731,0.363786,0.353446,0.300522,0.060567,0.245627,0.282447,0.059876,0.853396
시작,0.269515,0.185702,0.378862,1.000000,0.251415,0.339757,0.214191,0.255956,0.115805,0.180547,...,0.119984,0.183841,0.339584,0.330566,0.280831,0.080453,0.229462,0.264077,0.056091,0.278048
어땠을까 (Feat. 박정현),0.240874,0.190406,0.269188,0.251415,1.000000,0.253273,0.203681,0.262198,0.064289,0.196888,...,0.067751,0.140151,0.253178,0.244524,0.251207,0.053778,0.204917,0.251114,0.053263,0.263915


In [26]:
np.sort(cosine_sim_df.loc[:, "야생화"].values.reshape(1, -1).flatten())[::-1][1:11]

array([0.35302847, 0.34550273, 0.29082286, 0.28816015, 0.28304988,
       0.27779608, 0.27585678, 0.26951542, 0.2695035 , 0.26321945])

In [27]:
# Content Based Filtering

def get_recommendations(target_title, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_score = np.sort(matrix.loc[:, target_title].values.reshape(1, -1).flatten())[::-1][1:k+1]
    recom_title = items.iloc[recom_idx, :].title.values
    target_title_list = np.full(len(range(k)), target_title)
    d = {
        'target_title': target_title_list,
        'recom_title' : recom_title,
        'score': recom_score
    }
    return pd.DataFrame(d)

In [28]:
get_recommendations('야생화', cosine_sim_df, musics)

,target_title,recom_title,score
0,야생화,취중진담,0.353028
1,야생화,잠시만 안녕,0.345503
2,야생화,가지마 가지마,0.290823
3,야생화,너였다면,0.288160
4,야생화,오늘도 빛나는 너에게,0.283050
5,야생화,응급실,0.277796
6,야생화,Don't Cry,0.275857
7,야생화,시작,0.269515
8,야생화,부동의 첫사랑,0.269504
9,야생화,잘못된 만남,0.263219


In [29]:
get_recommendations('Lemon', cosine_sim_df, musics)

,target_title,recom_title,score
0,Lemon,NIGHT DANCER,0.600026
1,Lemon,사랑은 늘 도망가,0.190026
2,Lemon,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,0.189999
3,Lemon,부동의 첫사랑,0.181181
4,Lemon,너였다면,0.175427
5,Lemon,오늘도 빛나는 너에게,0.172082
6,Lemon,친구 얘긴데 (feat. Skinny Brown),0.171008
7,Lemon,"모든 날, 모든 순간",0.165821
8,Lemon,시작,0.163932
9,Lemon,취중진담,0.163314


In [30]:
get_recommendations('Spicy', cosine_sim_df, musics)

,target_title,recom_title,score
0,Spicy,퀸카 (Queencard),0.353739
1,Spicy,꽃,0.345890
2,Spicy,저곳으로(인어공주OST),0.316321
3,Spicy,FRIEND THE END,0.314196
4,Spicy,인형의 꿈,0.311497
5,Spicy,첫사랑,0.310609
6,Spicy,UNFORGIVEN (feat. Nile Rodgers),0.237948
7,Spicy,I AM,0.219743
8,Spicy,다시 만난 세계,0.199585
9,Spicy,잘못된 만남,0.092567


### Music ID 이용

In [31]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = musics.music_id, columns = musics.music_id)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(38, 38)


music_id,169,170,171,172,173,174,175,176,177,178,...,197,198,199,200,201,202,203,204,205,206
music_id,,,,,,,,,,,,,,,,,,,,,
169,1.000000,0.189866,0.288160,0.269515,0.240874,0.255166,0.275857,0.345503,0.051101,0.173269,...,0.054518,0.139127,0.255158,0.246726,0.269504,0.054152,0.220019,0.252769,0.084715,0.283050
170,0.189866,1.000000,0.198723,0.185702,0.190406,0.186918,0.160803,0.205978,0.034367,0.146019,...,0.036894,0.119835,0.186945,0.180638,0.185708,0.036646,0.161973,0.185424,0.036303,0.207573
171,0.288160,0.198723,1.000000,0.378862,0.269188,0.363342,0.247087,0.273992,0.123708,0.193486,...,0.128140,0.196731,0.363786,0.353446,0.300522,0.060567,0.245627,0.282447,0.059876,0.853396
172,0.269515,0.185702,0.378862,1.000000,0.251415,0.339757,0.214191,0.255956,0.115805,0.180547,...,0.119984,0.183841,0.339584,0.330566,0.280831,0.080453,0.229462,0.264077,0.056091,0.278048
173,0.240874,0.190406,0.269188,0.251415,1.000000,0.253273,0.203681,0.262198,0.064289,0.196888,...,0.067751,0.140151,0.253178,0.244524,0.251207,0.053778,0.204917,0.251114,0.053263,0.263915


In [32]:
def get_recommendations(target_id, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_id].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_score = np.sort(matrix.loc[:, target_id].values.reshape(1, -1).flatten())[::-1][1:k+1]
    recom_id = items.iloc[recom_idx, :].music_id.values
    target_title_list = np.full(len(range(k)), target_id)
    d = {
        'target_id': target_title_list,
        'recom_id' : recom_id,
        'score': recom_score
    }
    return pd.DataFrame(d)

result_sample = get_recommendations(169, cosine_sim_df, musics)

In [33]:
result_sample

,target_id,recom_id,score
0,169,180,0.353028
1,169,176,0.345503
2,169,182,0.290823
3,169,171,0.288160
4,169,206,0.283050
5,169,186,0.277796
6,169,175,0.275857
7,169,172,0.269515
8,169,201,0.269504
9,169,184,0.263219


In [34]:
from datetime import datetime

now = datetime.now()
version = now.strftime("%Y-%m-%d %H:%M:%S")
version

'2024-10-16 02:24:54'

In [35]:
from database import save_similar_music

save_similar_music(169, result_sample, version)